<a href="https://colab.research.google.com/github/eedduu/TFG/blob/main/code/utilsTFG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch.nn as nn
import numpy as np
import torch
import pandas as pd
import torch.nn.functional as F
import fastai
from fastai.vision.all import *
from fastai.metrics import *
# Set seed for fastai


# Set seed for torch
torch.manual_seed(42)
if torch.cuda.is_available():
  torch.cuda.manual_seed_all(42)

# Set seed for numpy
np.random.seed(42)

In [ ]:
#Resnet32 model

class BottleneckBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, bottleneck_ratio=4):
        super(BottleneckBlock, self).__init__()
        bottleneck_channels = out_channels // bottleneck_ratio

        self.conv1 = nn.Conv2d(in_channels, bottleneck_channels, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn1 = nn.BatchNorm2d(bottleneck_channels)

        self.conv2 = nn.Conv2d(bottleneck_channels, bottleneck_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(bottleneck_channels)

        self.conv3 = nn.Conv2d(bottleneck_channels, out_channels, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels)

        self.relu = nn.ReLU(inplace=True)

        self.downsample = None
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels),
            )

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

class ResNet57(nn.Module):
    def __init__(self, num_classes=1000, in_channels=1):
        super(ResNet57, self).__init__()
        self.in_channels = in_channels

        self.conv1 = nn.Conv2d(self.in_channels, 64, kernel_size=7, stride=1, bias=False)
        self.bn = nn.BatchNorm2d(64)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(64, 3, stride=1, input_channels=64)
        self.layer2 = self._make_layer(128, 4, stride=2)
        self.layer3 = self._make_layer(256, 4, stride=2)
        self.layer5 = self._make_layer(512, 3, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.bn1 = nn.BatchNorm1d(512)
        self.dropout1 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(512, num_classes)
        #self.bn2 = nn.BatchNorm1d(256)
        #self.dropout2 = nn.Dropout(0.5)
        #self.fc2 = nn.Linear(256, num_classes)
        self.relu = nn.ReLU(inplace=True)

    def _make_layer(self, out_channels, blocks, stride, input_channels=None):
        layers = []

        # Set input channels for the first block of the first layer
        if input_channels:
            self.in_channels = input_channels

        layers.append(BottleneckBlock(self.in_channels, out_channels, stride))
        self.in_channels = out_channels
        for _ in range(1, blocks):
            layers.append(BottleneckBlock(self.in_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):

        x = self.conv1(x)
        x = self.relu(x)
        x = self.bn(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer5(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.bn1(x)
        x = self.dropout1(x)
        x = self.fc1(x)
       # x = self.relu(x)
       # x = self.bn2(x)
       # x = self.dropout2(x)
       # x = self.fc2(x)


        return x



class ResNet15(nn.Module):
    def __init__(self, num_classes=1000, in_channels=1):
        super(ResNet15, self).__init__()
        self.in_channels = in_channels

        self.conv1 = nn.Conv2d(self.in_channels, 64, kernel_size=7, stride=1, bias=False)
        self.bn = nn.BatchNorm2d(64)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(64, 1, stride=1, input_channels=64)
        self.layer2 = self._make_layer(128, 1, stride=2)
        self.layer3 = self._make_layer(256, 1, stride=2)
        self.layer5 = self._make_layer(512, 1, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.bn1 = nn.BatchNorm1d(512)
        self.dropout1 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(512, num_classes)
        #self.bn2 = nn.BatchNorm1d(256)
        #self.dropout2 = nn.Dropout(0.5)
        #self.fc2 = nn.Linear(256, num_classes)
        self.relu = nn.ReLU(inplace=True)

    def _make_layer(self, out_channels, blocks, stride, input_channels=None):
        layers = []

        # Set input channels for the first block of the first layer
        if input_channels:
            self.in_channels = input_channels

        layers.append(BottleneckBlock(self.in_channels, out_channels, stride))
        self.in_channels = out_channels
        for _ in range(1, blocks):
            layers.append(BottleneckBlock(self.in_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):

        x = self.conv1(x)
        x = self.relu(x)
        x = self.bn(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer5(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.bn1(x)
        x = self.dropout1(x)
        x = self.fc1(x)
       # x = self.relu(x)
       # x = self.bn2(x)
       # x = self.dropout2(x)
       # x = self.fc2(x)


        return x


In [ ]:
#LeNet 5 model

class LeNet5(nn.Module):
  def __init__(self, in_channels=1, num_classes=10):
    super(LeNet5, self).__init__()

    self.in_channels = in_channels
    self.conv1 = nn.Conv2d(in_channels, 6, kernel_size=5, stride=1)
    self.bn1 =  nn.BatchNorm2d(6)
    self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1)
    self.bn2 =  nn.BatchNorm2d(16)


    self.fc1 = nn.Linear(400, 120)
    self.bn3 =  nn.BatchNorm1d(120)
    self.fc2 = nn.Linear(120, 84)
    self.bn4 =  nn.BatchNorm1d(84)
    self.fc3 = nn.Linear(84, num_classes)

    self.relu = nn.ReLU(inplace=True)

  def forward(self, x):
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.pool(x)
    x = self.conv2(x)
    x = self.bn2(x)
    x = self.relu(x)
    x = self.pool(x)
    x = torch.flatten(x, 1)
    x = self.fc1(x)
    x = self.bn3(x)
    x = self.relu(x)
    x = self.fc2(x)
    x = self.bn4(x)
    x = self.relu(x)
    x = self.fc3(x)
    return x

In [ ]:
from sklearn.metrics import f1_score, balanced_accuracy_score, r2_score

# Define a function to compute the F1 score
def f1_score_func(preds, targets):
  preds = preds.argmax(dim=-1)  # Convert predictions to class labels
  return f1_score(targets.cpu().numpy(), preds.cpu().numpy(), average='weighted')  # Calculate F1 score


def balanced_accuracy_func(preds, targs):
    preds_proba = preds.softmax(dim=-1).cpu().numpy()  # Convert logits to probabilities
    targs = targs.squeeze().cpu().numpy()  # Convert targets to numpy array

    # Convert probabilities to class predictions
    preds_classes = preds_proba.argmax(axis=-1)

    # Compute balanced accuracy score
    return balanced_accuracy_score(targs, preds_classes)


# Define the Adjusted R² function

def adjusted_r2_score(y_true, y_pred):
    n = y_true.size(0)  # Number of observations
    k = y_pred.size(1) if len(y_pred.size()) > 1 else 1  # Number of predictors
    r2 = r2_score(y_true.cpu().numpy(), y_pred.cpu().numpy())
    adj_r2 = 1 - (1 - r2) * (n - 1) / (n - k - 1)
    return adj_r2

# Create a custom metric for FastAI
class AdjustedR2Score(Metric):
    def __init__(self):
        self.preds = []
        self.targets = []

    def reset(self):
        self.preds, self.targets = [], []

    def accumulate(self, learn):
        preds, targets = learn.pred, learn.y
        # Check if preds and targets are not empty
        if len(preds) > 0 and len(targets) > 0:
            self.preds.append(preds)
            self.targets.append(targets)

    @property
    def value(self):
        # Check if preds and targets are not empty before concatenating
        if self.preds and self.targets:
            preds = torch.cat(self.preds)
            targets = torch.cat(self.targets)
            return adjusted_r2_score(targets, preds)
        else:
            return 0.0

In [ ]:
# Function to reduce dataset while maintaining class balance
def reduce_dataset(dataset, target_size):
    targets = np.array(dataset.targets)
    classes, class_counts = np.unique(targets, return_counts=True)
    samples_per_class = target_size // len(classes)

    reduced_indices = []
    for cls in classes:
        cls_indices = np.where(targets == cls)[0]
        sampled_indices = np.random.choice(cls_indices, samples_per_class, replace=False)
        reduced_indices.extend(sampled_indices)

    reduced_dataset = torch.utils.data.Subset(dataset, reduced_indices)
    return reduced_dataset

# Verify class balance
def verify_class_balance(dataset):
    targets = np.array(dataset.dataset.targets)[dataset.indices]
    classes, class_counts = np.unique(targets, return_counts=True)
    for cls, count in zip(classes, class_counts):
        print(f'Class {cls}: {count} samples')


In [ ]:
def compare_models(model1, model2):
    for param1, param2 in zip(model1.state_dict().items(), model2.state_dict().items()):
        if not torch.equal(param1[1], param2[1]):
            return False
    return True

In [ ]:
import torch.nn.init as init

def init_weights_glorot(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        init.xavier_uniform_(m.weight)
        if m.bias is not None:
            init.constant_(m.bias, 0)

In [ ]:
import matplotlib.pyplot as plt

def plot_learners_training(learners, names = [], title='Accuracy, Training and Validation Loss', metric=['accuracy'], lim=1):
    """
    Plots the training and validation loss curves for an array of fastai learners.

    Parameters:
    - learners: list of fastai Learner objects
    - metric: 'loss' to plot loss curves (default), or 'metric' to plot a specific metric
    """
    plt.figure(figsize=(15, 9))
    colors = ['blue', 'green', 'red', 'purple', 'orange']
    max_epochs = max(len(learn.recorder.values) for learn in learners)
    for idx, learn in enumerate(learners):
        color = colors[idx % len(colors)]  # Cycle through colors
        train_losses = []
        val_losses = []
        metrics = [[] for _ in range(len(metric))]
        for epoch in range(len(learn.recorder.values)):
          train_losses.append(learn.recorder.values[epoch][0])
          val_losses.append(learn.recorder.values[epoch][1])
          for met in range(len(metric)):
            metrics[met].append(learn.recorder.values[epoch][2+met])
          label_suffix = ''
        label_prefix = f' - {names[idx]}' if len(names) > 0 else f'Learner {idx + 1}'
        # Plot training loss
        plt.plot(train_losses, label=label_prefix + ' - Training Loss', color=color)
        # Plot validation loss
        plt.plot(val_losses, label=label_prefix + ' - Validation Loss', linestyle='--', color=color)

        # Plot metrics
        lines=['-.', ':']
        for idx, met in enumerate(metric):
          plt.plot(metrics[idx], label=label_prefix + f' - {met}', linestyle=lines[idx], color=color)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.xlim(0, max_epochs-1)
    plt.ylim(0,lim)

    # Adjust the plot area to make room for the legend
    plt.subplots_adjust(right=0.75)

    # Move legend outside of the plot
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
    plt.show()


In [ ]:
def plot_mh_mlp_training(train_losses, val_losses, accuracies,  layers, names=[], title='Training Metrics', metric = ['Accuracy'], lim=1,f1score=None):
    """
    Plots the training and validation loss curves along with accuracy for an array of models.

    Parameters:
    - train_losses: list of lists, where each sublist contains the training loss per epoch for a model
    - val_losses: list of lists, where each sublist contains the validation loss per epoch for a model
    - train_accuracies: list of lists, where each sublist contains the training accuracy per epoch for a model
    - val_accuracies: list of lists, where each sublist contains the validation accuracy per epoch for a model
    - names: list of names for the models (default is an empty list)
    - title: title of the plot (default is 'Training Metrics')
    """
    plt.figure(figsize=(12, 6))
    max_epochs = max(len(train_losses[layer]) for layer in layers)
    colors = ['blue', 'green', 'red', 'purple', 'orange']

    for idx,layer in enumerate(train_losses.keys()):
        color = colors[idx % len(colors)]  # Cycle through colors
        label_suffix = ''
        label_prefix = f' - {names[idx]}' if len(names) > 0 else f'Model {idx + 1}'



        # Plot training loss
        plt.plot(train_losses[layer], label=label_prefix + ' - Training Loss', color=color)

        # Plot validation loss
        plt.plot(val_losses[layer], label=label_prefix + ' - Validation Loss', linestyle='--', color=color)

        # Plot balanced accuracy
        plt.plot(accuracies[layer], label=label_prefix + ' - Balanced Accuracy', linestyle='-.', color=color)

        # Plot F1 Scores
       #plt.plot(f1score[layer], label=label_prefix + ' - F1 Score', linestyle=':', color=color)


    plt.xlabel('Epochs')
    plt.ylabel('Metrics')
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.xlim(0, max_epochs-1)
    plt.ylim(0,lim)
    # Adjust the plot area to make room for the legend
    plt.subplots_adjust(right=0.75)

    # Move legend outside of the plot
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
    plt.show()

In [ ]:
#Get params from a model in a numpy array

def get_params_from_model(model):
    return np.concatenate([v.cpu().detach().numpy().ravel() for v in model.parameters()])



#The params are set into the global model to compute the loss.
def set_params_to_model(params, model):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    offset = 0
    for param in model.parameters():
        param_size = param.numel()
        param.data = torch.tensor(params[offset:offset + param_size], dtype=param.data.dtype).view(param.size()).to(device)
        param.requires_grad = True
        offset += param_size

#Get the valid loss
def err_param_valid(params, model, dls, task='clas'):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)  # Move model to the device


    set_params_to_model(params, model)

    model.eval()
    #Option 1: calculate the error per each batch and make the mean

    loss =0.0

    if task == 'clas':
      func = F.cross_entropy
    elif task == 'reg':
      func = F.mse_loss

    #If i have categorical, continius and the target
    if len(dls.one_batch()) > 2:
      for batch in dls.valid:
        # Handle the case where there are no continuous features
        x_cont = batch[1].to(device) if len(batch) > 1 and batch[1] is not None else None
        pred = model(batch[0].to(device), x_cont)  # Pass both categorical and continuous features (if any)
        pred = pred.squeeze().clone().detach() if task == 'reg' else pred.clone().detach()
        target = batch[2].to(device).squeeze().long().clone().detach()
        loss += func(pred, target)


    #If not
    else:
      for batch in dls.valid:
        # Handle the case where there are no continuous features

        pred = model(batch[0].to(device))  # Pass both categorical and continuous features (if any)
        pred = pred.squeeze().clone().detach() if task == 'reg' else pred.clone().detach()
        target = batch[1].to(device).squeeze().long().clone().detach()
        loss += func(pred, target)

    loss/=len(dls.valid)


    return loss.mean().item()



#Get de train loss
def err_param(params, model, dls, mode='train', task='clas'):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)  # Move model to the device
    set_params_to_model(params, model)

    if mode == 'train':
      model.train()
    else:
      model.eval()
    #Option 1: calculate the error per each batch and make the mean

    if task == 'clas':
      func = F.cross_entropy
    elif task == 'reg':
      func = F.mse_loss


    loss =0.0
    it = dls.train if hasattr(dls,'train') else dls
    #If i have categorical, continius and the target
    if len(dls.one_batch()) > 2:
      for batch in it:
        # Handle the case where there are no continuous features
        x_cont = batch[1].to(device) if len(batch) > 1 and batch[1] is not None else None
        pred = model(batch[0].to(device), x_cont)  # Pass both categorical and continuous features (if any)
        pred = pred.squeeze().clone().detach() if task == 'reg' else pred.clone().detach()
        target = batch[2].to(device).squeeze().long().clone().detach()
        loss += func(pred, target)


    #If not
    else:
      for batch in it:
        # Handle the case where there are no continuous features

        pred = model(batch[0].to(device))  # Pass both categorical and continuous features (if any)
        pred = pred.squeeze().clone().detach() if task == 'reg' else pred.clone().detach()
        target = batch[1].to(device).squeeze().long().clone().detach()
        loss += func(pred, target)

    loss/=len(it)


    #If i only have one element it wont affect, but if i have more its necessary
    return loss.mean().item()



def err_param_w_model(model, dls, mode='train', task='clas'):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)  # Move model to the device


    if mode == 'train':
      model.train()
    else:
      model.eval()

    if task == 'clas':
      func = F.cross_entropy
    elif task == 'reg':
      func = F.mse_loss


    #Option 1: calculate the error per each batch and make the mean

    loss =0.0

    #If i have categorical, continius and the target
    it = dls.train if hasattr(dls,'train') else dls
    if len(dls.one_batch()) > 2:


      for batch in it:
        # Handle the case where there are no continuous features
        x_cont = batch[1].to(device) if len(batch) > 1 and batch[1] is not None else None
        pred = model(batch[0].to(device), x_cont)  # Pass both categorical and continuous features (if any)
        pred = pred.squeeze().clone().detach() if task == 'reg' else pred.clone().detach()
        target = batch[2].to(device).squeeze().long().clone().detach()
        loss += func(pred, target)


    #If not
    else:
      for batch in it:
        # Handle the case where there are no continuous features

        pred = model(batch[0].to(device))  # Pass both categorical and continuous features (if any)
        pred = pred.squeeze().clone().detach() if task == 'reg' else pred.clone().detach()
        target = batch[1].to(device).squeeze().long().clone().detach()
        loss += func(pred, target)

    loss/=len(it)


    #If i only have one element it wont affect, but if i have more its necessary
    return loss.mean().item()

def err_param_shade2(params, opts):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model, dls, task = opts
    model = model.to(device)  # Move model to the device
    set_params_to_model(params, model)

    if task == 'clas':
      func = F.cross_entropy
    elif task == 'reg':
      func = F.mse_loss

    model.train()
    #Option 1: calculate the error per each batch and make the mean

    loss =0.0

    #If i have categorical, continius and the target
    if len(dls.one_batch()) > 2:
      for batch in dls.train:
        # Handle the case where there are no continuous features
        x_cont = batch[1].to(device) if len(batch) > 1 and batch[1] is not None else None
        pred = model(batch[0].to(device), x_cont)  # Pass both categorical and continuous features (if any)
        pred = pred.squeeze().clone().detach() if task == 'reg' else pred.clone().detach()
        target = batch[2].to(device).squeeze().long().clone().detach()
        loss += func(pred, target)


    #If not
    else:
      for batch in dls.train:
        # Handle the case where there are no continuous features

        pred = model(batch[0].to(device))  # Pass both categorical and continuous features (if any)
        pred = pred.squeeze().clone().detach() if task == 'reg' else pred.clone().detach()
        target = batch[1].to(device).squeeze().long().clone().detach()
        loss += func(pred, target)

    loss/=len(dls.train)


    #If i only have one element it wont affect, but if i have more its necessary
    return loss.mean().item()




#def test_err_param(params, model, batch):
#    set_params_to_model(params, model)
#
#    model.train()
#    #Option 1: calculate the error per each batch and make the mean
#
#    # Handle the case where there are no continuous features
#    if len(batch) > 2:
#      x_cont = batch[1] if len(batch) > 1 and batch[1] is not None else None
#      pred = model(batch[0], x_cont)  # Pass both categorical and continuous features (if any)
#      pred = torch.tensor(pred.clone().detach())#.to('cuda')
#      target = torch.tensor(batch[2].squeeze().long().clone().detach())#.to('cuda')
#
#    else:
#      pred = model(batch[0])  # Pass both categorical and continuous features (if any)
#      pred = torch.tensor(pred.clone().detach())#.to('cuda')
#      target = torch.tensor(batch[1].squeeze().long().clone().detach())#.to('cuda')
#
#    loss = F.cross_entropy(pred, target)
#
#
#
#    return loss.mean().item()




In [ ]:
#BUSQUEDA LOCAL CON LBFGS CON LIBRERIA SCIPY
from scipy.optimize import minimize
from functools import partial
def get_gradient(ind, model, dls, task):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)  # Move model to the device

    set_params_to_model(ind, model)
    model.train()  # Put the model in training mode
    batch = dls.train.one_batch()

    if task == 'clas':
      func = F.cross_entropy
    elif task == 'reg':
      func = F.mse_loss


    if len(batch) > 2:
      x_cont = batch[1].to(device) if len(batch) > 1 and batch[1] is not None else None
      pred = model(batch[0].to(device), x_cont)  # Make predictions
      pred = pred.as_subclass(torch.Tensor).clone()
      target = batch[2].to(device).squeeze().long().as_subclass(torch.Tensor).clone()

    else:
      pred = model(batch[0].to(device))  # Make predictions
      # Convert predictions and targets to torch.Tensor subclass
      pred = pred.as_subclass(torch.Tensor).clone()
      target = batch[1].to(device).squeeze().long().as_subclass(torch.Tensor).clone()

    # Calculate the loss using cross_entropy
    loss = func(pred, target)
    loss.backward()  # Calculate gradients

    # Extract gradients into a list
    gradients = [param.grad.clone().detach().cpu().numpy().ravel() for param in model.parameters()]

    # Concatenate gradients into a single 1D array
    return loss.mean().item(), np.concatenate(gradients)


def get_gradient_full(ind, model, dls, task):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)  # Move model to the device
    set_params_to_model(ind, model)

    model.train()

    if task == 'clas':
      func = F.cross_entropy
    elif task == 'reg':
      func = F.mse_loss


    model.zero_grad()
    #Option 1: calculate the error per each batch and make the mean

    loss =0.0

    #If i have categorical, continius and the target
    if len(dls.one_batch()) > 2:
      for batch in dls.train:
        # Handle the case where there are no continuous features
        x_cont = batch[1].to(device) if len(batch) > 1 and batch[1] is not None else None
        pred = model(batch[0].to(device), x_cont)  # Pass both categorical and continuous features (if any)
        pred = pred.squeeze().clone() if task == 'reg' else pred.clone()
        target = batch[2].to(device).squeeze().float().clone() if task == 'reg' else batch[2].to(device).squeeze().long().clone()
        loss += func(pred, target).float()


    #If not
    else:
      for batch in dls.train:
        # Handle the case where there are no continuous features

        pred = model(batch[0].to(device))  # Pass both categorical and continuous features (if any)
        pred = pred.clone()
        target = batch[1].to(device).squeeze().float().clone() if task == 'reg' else batch[1].to(device).squeeze().long().clone()
        loss += func(pred, target)

    loss/=len(dls.train)

    loss.backward()  # Calculate gradients

    # Extract gradients into a list
    gradients = [param.grad.clone().detach().cpu().numpy().ravel() for param in model.parameters()]

    # Concatenate gradients into a single 1D array
    return loss.item(), np.concatenate(gradients)

def LBFGS_SCIPY(ind, model, dls, task='clas'):
    get_grad = partial(get_gradient, model=model, dls=dls, task=task)
    result = minimize(get_grad, ind, jac=True, method='L-BFGS-B')
    return result.x, result.fun, result.nfev, result.njev

def LBFGS_SCIPY_full(ind, model, dls, max=5, task='clas'):
  get_grad = partial(get_gradient_full, model=model, dls=dls, task=task)
  result = minimize(get_grad, ind, jac=True, method='L-BFGS-B', options={'maxiter': max})
  return result.x, result.fun, result.nfev, result.njev


In [ ]:
from typing import Callable
!pip install /content/drive/MyDrive/pyade-master
import pyade

import pyade.shade

algorithm = pyade.shade

def SHADE_ej(population, fitness, pop_size, max_evals, dim, dls, prevm_cr=0, prevm_f=0, k=0, model=None, task='clas'):

  population = np.array(population).reshape(pop_size, dim)
  fitness = np.array(fitness)

  params = algorithm.get_default_params(dim=dim)
  # We define the boundaries of the variables
  params['bounds'] = np.array([[-50, 50]] * dim)

  # We indicate the function we want to minimize
  params['func'] = err_param_shade

  params['population_size']=pop_size



  params['seed']=42

  params['opts']=[model, dls, task]




  params['init_pop']=population

  params['max_evals']=max_evals


  params['prevm_cr']=prevm_cr

  params['prevm_f']=prevm_f

  params['k']=k

  params['fitness']=fitness

  return algorithm.apply(**params)

Processing ./drive/MyDrive/pyade-master
  Preparing metadata (setup.py) ... done
  Created wheel for pyade-python: filename=pyade_python-1.0-py3-none-any.whl size=29072 sha256=07541f727ee91604b9a3adafa755333834bb36844754f5087783c6a055049262
  Stored in directory: /root/.cache/pip/wheels/89/b8/51/ff2ebef24405d3ef46ccb451f962b374b2745f263fb000c434
Successfully built pyade-python


In [ ]:
#Algoritmo SHADE-ILS
from collections import deque

import copy

def reset_weights(m):
    if hasattr(m, 'reset_parameters'):
        m.reset_parameters()



def SHADE_ILS(population, fit, max_evals, max_shade, dls, learner, model,  max_ls=5, func=None, task='clas'):
  LEARN=learner


  historic_fitness=[]
  historic_best_solution=[]

  population_size = len(population)
  evals = population_size
  max_evals = max_evals
  max_evals_SHADE = max_shade
  prevm_cr=0
  prevm_f=0
  k=0
  must_restart=False
  improq = deque([1,1,1], maxlen=3)

  #Initialize the population
  #population = [get_params_from_model(model) for _ in range(len(population))]
  fitness = fit
  size_ind=len(population[0])

  #Select the best

  current_best_fitness = min(fitness)
  current_best_index = fitness.index(current_best_fitness)
  current_best = population[current_best_index]

  historic_fitness.append(current_best_fitness)
  historic_best_solution.append(current_best)

  temp_current_best, _, e1, e2 = LBFGS_SCIPY_full(current_best, model, dls, max=max_ls, task=task)
  evals += e1
  temp_current_best_fitness=err_param(temp_current_best, model, dls, task=task)

  improq.append((current_best_fitness-temp_current_best_fitness)/current_best_fitness)

  if temp_current_best_fitness < current_best_fitness:
    current_best_fitness = temp_current_best_fitness
    current_best = temp_current_best


  population[current_best_index] = current_best
  fitness[current_best_index] = current_best_fitness

  best_solution=current_best
  best_fitness=current_best_fitness
  historic_fitness.append(best_fitness)
  historic_best_solution.append(best_solution)

  while evals < max_evals:
    print("eo")
    population, fitness, prevm_cr, prevm_f, k , _, _= SHADE_ej(population, fitness, population_size, max_evals_SHADE, size_ind, dls,  prevm_cr=prevm_cr, prevm_f=prevm_f, k=k, model=model, task=task)
    evals += max_evals_SHADE
    temp_current_best_fitness= min(fitness)

    improq.append((current_best_fitness-temp_current_best_fitness)/current_best_fitness)

    if temp_current_best_fitness < current_best_fitness:
      current_best_fitness = temp_current_best_fitness
      current_best_index = list(fitness).index(current_best_fitness)
      current_best = population[current_best_index]



    print(improq)
    if np.all(np.array(improq)<0.05):
      must_restart=True

    #Choose the LS method to apply this iteration based on improvement

    temp_current_best, _, e1, e2 = LBFGS_SCIPY_full(current_best, model, dls, max=max_ls, task=task)
    evals += e1
    temp_current_best_fitness=err_param(temp_current_best, model, dls, task=task)

    improq.append((current_best_fitness-temp_current_best_fitness)/current_best_fitness)

    #if temp_current_best_fitness < current_best_fitness:
    current_best_fitness = temp_current_best_fitness
    current_best = temp_current_best


    print(improq)
    if np.all(np.array(improq)<0.05):
      must_restart=True

    population[current_best_index] = current_best
    fitness[current_best_index] = current_best_fitness

    #Update the probability to apply LS in next iterations

    if current_best_fitness < best_fitness:
      best_fitness = current_best_fitness
      best_solution = copy.deepcopy(current_best)

    historic_fitness.append(best_fitness)
    historic_best_solution.append(best_solution)

    if must_restart:
      random_index = np.random.choice(population_size)
      sol = population[random_index]
      sol += np.random.normal(0, 0.5, len(sol))
      k=0
      prevm_cr=0
      prevm_f=0
      population=[]
      for _ in range(population_size-1):
        model_copy = copy.deepcopy(model)

        # Reset the weights of the model copy
        model_copy.apply(init_weights_glorot)

        # Add the reinitialized model to the population list
        population.append(get_params_from_model(model_copy))

      fitness = [err_param(elem, model, dls,task=task) for elem in population]
      evals += population_size
      population.append(sol)
      current_best_fitness=err_param(sol, model, dls,task=task)
      current_best=sol
      fitness.append(current_best_fitness)
      improq=deque([1,1,1], maxlen=3)

      print("Reiniciado")
      must_restart=False

    print(evals)
  return best_solution, best_fitness, historic_fitness, historic_best_solution



def SHADE_GD(population, fit, max_evals, max_shade, dls, learner, model, task='clas'):
  LEARN=learner


  historic_fitness=[]
  historic_best_solution=[]

  population_size = len(population)
  evals = population_size
  max_evals = max_evals
  max_evals_SHADE = max_shade
  prevm_cr=0
  prevm_f=0
  k=0
  must_restart=False
  improq = deque([1,1,1], maxlen=3)

  app_gener= max_evals/max_evals_SHADE
  gen=0

  #Initialize the population
  #population = [get_params_from_model(model) for _ in range(len(population))]
  fitness = fit
  evals += population_size
  size_ind=len(population[0])

  #Select the best

  current_best_fitness = min(fitness)
  current_best_index = fitness.index(current_best_fitness)
  current_best = population[current_best_index]

  population[current_best_index] = current_best
  fitness[current_best_index] = current_best_fitness

  best_solution=current_best
  best_fitness=current_best_fitness
  historic_fitness.append(best_fitness)
  historic_best_solution.append(best_solution)

  while evals < max_evals:
    gen+=1
    print("eo")
    population, fitness, prevm_cr, prevm_f, k, _, _ = SHADE_ej(population, fitness, population_size, max_evals_SHADE, size_ind, dls,  prevm_cr=prevm_cr, prevm_f=prevm_f, k=k, model=model, task=task)
    evals += max_evals_SHADE
    temp_current_best_fitness= min(fitness)

    improq.append((current_best_fitness-temp_current_best_fitness)/current_best_fitness)

    if temp_current_best_fitness < current_best_fitness:
      current_best_fitness = temp_current_best_fitness
      current_best_index = list(fitness).index(current_best_fitness)
      current_best = population[current_best_index]



    print(improq)
    if np.all(np.array(improq)<0.05):
      must_restart=True

    #Choose the LS method to apply this iteration based on improvement


    if gen%5==0:
      ind = np.random.randint(0, population_size)
      set_params_to_model(population[ind], LEARN.model)
      try:
        lr=LEARN.lr_find()

      except E as e:
        print(f"Ha fallado {e} con ind={ind}, current_best_fitness={current_best_fitness} y best_fitness={best_fitness}")

      finally:
        LEARN.fit(1, lr)
      population[ind]=get_params_from_model(LEARN.model)
      fit=LEARN.recorder.values[-1][0]
      fitness[ind]=fit
      temp_current_best_fitness=fit
      temp_current_best=population[ind]
      evals+=1


      improq.append((current_best_fitness-temp_current_best_fitness)/current_best_fitness)

      #if temp_current_best_fitness < current_best_fitness:
      current_best_fitness = temp_current_best_fitness
      current_best = temp_current_best


    print(improq)
    if np.all(np.array(improq)<0.05):
      must_restart=True



    #Update the probability to apply LS in next iterations

    if current_best_fitness < best_fitness:
      best_fitness = np.copy(current_best_fitness)
      best_solution = np.copy(current_best)

    historic_fitness.append(best_fitness if np.isscalar(best_fitness) else float(best_fitness))
    historic_best_solution.append(best_solution)

    if must_restart:
      random_index = np.random.choice(population_size)
      sol = population[random_index]
      sol += np.random.normal(0, 0.3, len(sol))

      k=0
      prevm_cr=0
      prevm_f=0
      population=[]
      for _ in range(population_size-1):
        model_copy = copy.deepcopy(model)

        # Reset the weights of the model copy
        model_copy.apply(init_weights_glorot)

        # Add the reinitialized model to the population list
        population.append(get_params_from_model(model_copy))

      fitness = [err_param(elem, model, dls,task=task) for elem in population]
      evals += population_size
      population.append(sol)
      current_best_fitness=err_param(sol, model, dls, task=task)
      current_best=sol
      fitness.append(current_best_fitness)
      improq=deque([1,1,1], maxlen=3)

      print("Reiniciado")
      must_restart=False

  return best_solution, best_fitness, historic_fitness, historic_best_solution




def SHADE_ILS_GD(population, fit, max_evals, max_shade, dls, learner, model, max_ls=5, task='clas'):
  LEARN=learner


  historic_fitness=[]
  historic_best_solution=[]

  population_size = len(population)
  evals = population_size
  max_evals = max_evals
  max_evals_SHADE = max_shade
  prevm_cr=0
  prevm_f=0
  k=0
  lr=0.001
  must_restart=False
  improq = deque([1,1,1], maxlen=3)

  app_gener= max_evals/max_evals_SHADE
  gen=0

  #Initialize the population
  #population = [get_params_from_model(model) for _ in range(len(population))]
  population = population
  fitness = fit
  evals += population_size
  size_ind=len(population[0])

  #Select the best

  current_best_fitness = min(fitness)
  current_best_index = fitness.index(current_best_fitness)
  current_best = population[current_best_index]

  population[current_best_index] = current_best
  fitness[current_best_index] = current_best_fitness

  best_solution=current_best
  best_fitness=current_best_fitness
  historic_fitness.append(best_fitness)
  historic_best_solution.append(best_solution)

  temp_current_best, _, e1, e2 = LBFGS_SCIPY_full(current_best, model, dls, max=max_ls, task=task)
  evals += e1
  temp_current_best_fitness=err_param(temp_current_best, model, dls,task=task)

  improq.append((current_best_fitness-temp_current_best_fitness)/current_best_fitness)

  if temp_current_best_fitness < current_best_fitness:
    current_best_fitness = temp_current_best_fitness
    current_best = temp_current_best


  print(improq)
  if np.all(np.array(improq)<0.05):
    must_restart=True

  population[current_best_index] = current_best
  fitness[current_best_index] = current_best_fitness


  historic_fitness.append(current_best_fitness)
  historic_best_solution.append(current_best)

  while evals < max_evals:
    gen+=1
    print("eo")
    population, fitness, prevm_cr, prevm_f, k, _, _ = SHADE_ej(population, fitness, population_size, max_evals_SHADE, size_ind, dls,  prevm_cr=prevm_cr, prevm_f=prevm_f, k=k, model=model, task=task)
    evals += max_evals_SHADE
    temp_current_best_fitness= min(fitness)

    improq.append((current_best_fitness-temp_current_best_fitness)/current_best_fitness)

    if temp_current_best_fitness < current_best_fitness:
      current_best_fitness = temp_current_best_fitness
      current_best_index = list(fitness).index(current_best_fitness)
      current_best = population[current_best_index]



    print(improq)
    if np.all(np.array(improq)<0.05):
      must_restart=True


    temp_current_best, _, e1, e2 = LBFGS_SCIPY_full(current_best, model, dls, max=max_ls, task=task)
    evals += e1
    temp_current_best_fitness=err_param(temp_current_best, model, dls, task=task)

    improq.append((current_best_fitness-temp_current_best_fitness)/current_best_fitness)

    #if temp_current_best_fitness < current_best_fitness:
    current_best_fitness = temp_current_best_fitness
    current_best = temp_current_best


    print(improq)
    if np.all(np.array(improq)<0.05):
      must_restart=True

    population[current_best_index] = current_best
    fitness[current_best_index] = current_best_fitness


    if gen%5==0:
      ind = np.random.randint(0, population_size)
      set_params_to_model(population[ind], LEARN.model)

      try:
        lr=LEARN.lr_find()

      except E as e:
        print(f"Ha fallado {e} con ind={ind}, current_best_fitness={current_best_fitness} y best_fitness={best_fitness}")

      finally:
        LEARN.fit(1, lr)
      population[ind]=get_params_from_model(LEARN.model)
      fit=LEARN.recorder.values[-1][0]
      fitness[ind]=fit
      temp_current_best_fitness=fit
      temp_current_best=population[ind]
      evals+=1


    improq.append((current_best_fitness-temp_current_best_fitness)/current_best_fitness)

    #if temp_current_best_fitness < current_best_fitness:
    current_best_fitness = temp_current_best_fitness
    current_best = temp_current_best


    print(improq)
    if np.all(np.array(improq)<0.05):
      must_restart=True



    #Update the probability to apply LS in next iterations

    if current_best_fitness < best_fitness:
      best_fitness = np.copy(current_best_fitness)
      best_solution = np.copy(current_best)

    historic_fitness.append(best_fitness if np.isscalar(best_fitness) else float(best_fitness))
    historic_best_solution.append(best_solution)

    if must_restart:
      random_index = np.random.choice(population_size)
      sol = population[random_index]
      sol += np.random.normal(0, 0.3, len(sol))

      k=0
      prevm_cr=0
      prevm_f=0
      population=[]
      for _ in range(population_size-1):
        model_copy = copy.deepcopy(model)

        # Reset the weights of the model copy
        model_copy.apply(init_weights_glorot)

        # Add the reinitialized model to the population list
        population.append(get_params_from_model(model_copy))
      fitness = [err_param(elem, model, dls,task=task) for elem in population]
      evals += population_size
      population.append(sol)
      current_best_fitness=err_param(sol, model, dls,task=task)
      current_best=sol
      fitness.append(current_best_fitness)
      improq=deque([1,1,1], maxlen=3)

      print("Reiniciado")
      must_restart=False

  return best_solution, best_fitness, historic_fitness, historic_best_solution


In [ ]:
#GenAlg

# Crossover operator for two childs, both being arrays of parameters
#def crossover(parent1, parent2):
#  mask = np.random.choice([True, False], size=len(parent1), p=[0.5, 0.5])
#  child1 = np.where(mask, parent1, parent2)
#  child2 = np.where(mask, parent2, parent1)

#  return child1, child2

#Operator: choose one entire layer from one of the parents
def crossover(parent1, parent2, model):
    offset = 0
    child1=[]
    child2=[]
    for param in model.parameters():
        param_size = param.numel()
        #Generate a random integer 0 or 1
        random_int = np.random.randint(0, 2)
        if random_int == 0:
            child1[offset:offset + param_size] = parent1[offset:offset + param_size]
            child2[offset:offset + param_size] = parent2[offset:offset + param_size]
        else:
            child1[offset:offset + param_size] = parent2[offset:offset + param_size]
            child2[offset:offset + param_size] = parent1[offset:offset + param_size]
        offset += param_size

    return child1, child2

#operador bl-alpha

def bl_crossover(parent1, parent2, model, alpha=0.2):
    child1=[]
    child2=[]
    offset=0

    for param in model.parameters():
        param_size = param.numel()
        # Compute the distance between parent genes
        d = np.abs(parent1[offset:offset + param_size] - parent2[offset:offset + param_size])

        # Calculate the lower and upper bounds for the offspring genes
        lower_bound = np.minimum(parent1[offset:offset + param_size], parent2[offset:offset + param_size]) - alpha * d
        upper_bound = np.maximum(parent1[offset:offset + param_size], parent2[offset:offset + param_size]) + alpha * d

        # Generate a random offspring within the specified bounds
        child1[offset:offset + param_size] = np.random.uniform(lower_bound, upper_bound)
        child2[offset:offset + param_size] = np.random.uniform(lower_bound, upper_bound)

        offset += param_size

    return child1, child2


def mutate(child, mutation_rate=0.01, mutation_std=0.1):

    if isinstance(child, list):
        child = np.array(child)
    # Generate a mask where mutation_rate probability of mutation
    mask = np.random.rand(*child.shape) < mutation_rate

    # Generate mutation values with normal distribution
    mutation = np.random.normal(loc=0.0, scale=mutation_std, size=child.shape)

    # Apply mutation: element-wise where mask is True
    mutated_child = np.where(mask, child + mutation, child)

    return mutated_child



def gen_alg(pop_size, population, fit, dls, max_evals, learner):

  LEARN = learner

  population = population
  fitness = fit
  evals=pop_size
  historic_fitness=[np.min(fitness)]

  while evals < max_evals:
    # Selection (tournament selection)

    #Select 2 groups of 3 individuals
    selected_indices_1 = np.random.choice(range(pop_size), size=3, replace=False)
    selected_indices_2 = np.random.choice(range(pop_size), size=3, replace=False)


    # Get fitness scores for each group
    selected_fitness_scores_1 = [fitness[i] for i in selected_indices_1]
    selected_fitness_scores_2 = [fitness[i] for i in selected_indices_2]

    # Find the best individual in each group
    best1_index = selected_indices_1[np.argmin(selected_fitness_scores_1)] # Find the index of the best individual within the selected group
    best2_index = selected_indices_2[np.argmin(selected_fitness_scores_2)]
    parent1 = population[best1_index]
    parent2 = population[best2_index]

    # Crossover
    child1, child2 = crossover(parent1, parent2, LEARN.model)


    # Repeat the process to get another child
    selected_indices_1 = np.random.choice(range(pop_size), size=3, replace=False)
    selected_indices_2 = np.random.choice(range(pop_size), size=3, replace=False)

    selected_fitness_scores_1 = [fitness[i] for i in selected_indices_1]
    selected_fitness_scores_2 = [fitness[i] for i in selected_indices_2]

    best1_index = selected_indices_1[np.argmin(selected_fitness_scores_1)] # Find the index of the best individual within the selected group
    best2_index = selected_indices_2[np.argmin(selected_fitness_scores_2)]

    parent1 = population[best1_index]
    parent2 = population[best2_index]


    # Crossover
    child3, child4 = crossover(parent1, parent2, LEARN.model)


    #Mutation and getting fitness
    children = [child1, child2, child3, child4]
    children = [mutate(child) for child in children]
    fitness_children=[err_param(child, LEARN.model, dls, task=task) for child in children]
    evals+=4


    #Getting best two children
    best_two_children = np.argsort(fitness_children)[:2]
    best_child1 = children[best_two_children[0]]
    best_child2 = children[best_two_children[1]]
    best_fitness_child1 = fitness_children[best_two_children[0]]
    best_fitness_child2 = fitness_children[best_two_children[1]]

    #Compare two worst individuals in the population with the best children
    worst_indices = np.argsort(fitness)[-2:]
    worst_individual1 = population[worst_indices[0]]
    worst_individual2 = population[worst_indices[1]]
    worst_fitness1 = fitness[worst_indices[0]]
    worst_fitness2 = fitness[worst_indices[1]]


    #Get the best two candidates and insert them in the population
    candidates = [worst_individual1, worst_individual2, best_child1, best_child2]
    candidates_fitness = [best_fitness_child1, best_fitness_child2, worst_fitness1, worst_fitness2]

    best_two_candidates = np.argsort(candidates_fitness)[:2]
    best_candidate1 = candidates[best_two_candidates[0]]
    best_candidate2 = candidates[best_two_candidates[1]]

    population[worst_indices[0]] = best_candidate1
    population[worst_indices[1]] = best_candidate2

    historic_fitness.append(np.min(fitness))


  return population[np.argmin(fitness)], np.min(fitness), historic_fitness



In [ ]:
#MemAlg

def mem_alg(pop_size, population, dls, max_evals, learner):

  LEARN= learner

  fitness = [err_param(elem, LEARN.model, dls, task=task) for elem in population]
  evals=pop_size
  historic_fitness=[np.min(fitness)]
  generations = (max_evals-evals) // 4
  it= generations//10
  gen=1

  while evals < max_evals and gen<generations:
    # Selection (tournament selection)

    #Select 2 groups of 3 individuals
    selected_indices_1 = np.random.choice(range(pop_size), size=3, replace=False)
    selected_indices_2 = np.random.choice(range(pop_size), size=3, replace=False)


    # Get fitness scores for each group
    selected_fitness_scores_1 = [fitness[i] for i in selected_indices_1]
    selected_fitness_scores_2 = [fitness[i] for i in selected_indices_2]

    # Find the best individual in each group
    best1_index = selected_indices_1[np.argmin(selected_fitness_scores_1)] # Find the index of the best individual within the selected group
    best2_index = selected_indices_2[np.argmin(selected_fitness_scores_2)]
    parent1 = population[best1_index]
    parent2 = population[best2_index]

    # Crossover
    child1, child2 = crossover(parent1, parent2, LEARN.model)


    # Repeat the process to get another child
    selected_indices_1 = np.random.choice(range(pop_size), size=3, replace=False)
    selected_indices_2 = np.random.choice(range(pop_size), size=3, replace=False)

    selected_fitness_scores_1 = [fitness[i] for i in selected_indices_1]
    selected_fitness_scores_2 = [fitness[i] for i in selected_indices_2]

    best1_index = selected_indices_1[np.argmin(selected_fitness_scores_1)] # Find the index of the best individual within the selected group
    best2_index = selected_indices_2[np.argmin(selected_fitness_scores_2)]

    parent1 = population[best1_index]
    parent2 = population[best2_index]


    # Crossover
    child3, child4 = crossover(parent1, parent2, LEARN.model)


    #Mutation and getting fitness
    children = [child1, child2, child3, child4]
    children = [mutate(child) for child in children]
    fitness_children=[err_param(child, LEARN.model, dls, task=task) for child in children]
    evals+=4


    #Getting best two children
    best_two_children = np.argsort(fitness_children)[:2]
    best_child1 = children[best_two_children[0]]
    best_child2 = children[best_two_children[1]]
    best_fitness_child1 = fitness_children[best_two_children[0]]
    best_fitness_child2 = fitness_children[best_two_children[1]]

    #Compare two worst individuals in the population with the best children
    worst_indices = np.argsort(fitness)[-2:]
    worst_individual1 = population[worst_indices[0]]
    worst_individual2 = population[worst_indices[1]]
    worst_fitness1 = fitness[worst_indices[0]]
    worst_fitness2 = fitness[worst_indices[1]]


    #Get the best two candidates and insert them in the population
    candidates = [worst_individual1, worst_individual2, best_child1, best_child2]
    candidates_fitness = [best_fitness_child1, best_fitness_child2, worst_fitness1, worst_fitness2]

    best_two_candidates = np.argsort(candidates_fitness)[:2]
    best_candidate1 = candidates[best_two_candidates[0]]
    best_candidate2 = candidates[best_two_candidates[1]]

    population[worst_indices[0]] = best_candidate1
    population[worst_indices[1]] = best_candidate2

    if gen%it==0:
      ind = np.random.randint(0, pop_size)
      set_params_to_model(population[ind], LEARN.model)
      lr=LEARN.lr_find()
      LEARN.fit(1, lr)
      population[ind]=get_params_from_model(LEARN.model)
      fitness[ind]=LEARN.recorder.values[-1][0]
      evals+=1

    historic_fitness.append(np.min(fitness))
    gen+=1

  return population[np.argmin(fitness)], np.min(fitness), historic_fitness

